# Lab 3 - n Puzzle

In [1]:
import numpy as np
import random
from tqdm.auto import tqdm
import time
import itertools
from operator import itemgetter

In [2]:
class nPuzzle():

    def __init__(self,d,n_shuffle=500):
        self.d = d
        val = np.arange(1,self.d**2)
        self.n = n_shuffle
        self.solution = np.append(val,0).reshape(self.d,self.d)
        self.board = self.solution.copy()
        self.puzzle_shuffle(self.n)

    def __str__(self):
        return f'{self.board}'
        
    def is_solved(self):
        if len(self.board)>0:
            return (self.board==self.solution).all()
        else:
            return False
        
    def find_void(self):
        pos = np.where(self.board==0)
        return (pos[0].item(), pos[1].item())
    
    def find_moves(self):
        r,c = self.find_void()
        
        if r in range(1,self.d-1) and c in range(1,self.d-1): # void cell not in contour
            return [(r,c-1),(r-1,c),(r+1,c),(r,c+1)]
        
        elif r in range(1,self.d-1) and c==0: # first column, middle rows
            return [(r-1,c),(r+1,c),(r,c+1)]
        
        elif r in range(1,self.d-1) and c==self.d-1: # last column, middle rows
            return [(r,c-1),(r-1,c),(r+1,c)]
        
        elif r==0 and c in range(1,self.d-1): # first row, middle columns
            return [(r,c-1),(r+1,c),(r,c+1)]
        
        elif r==self.d-1 and c in range(1,self.d-1): # last row, middle columns
            return [(r,c-1),(r-1,c),(r,c+1)]
        
        elif r==0 and c==0: # upper left corner
            return [(r+1,c),(r,c+1)]
        
        elif r==0 and c==self.d-1: # upper right corner
            return [(r+1,c),(r,c-1)]
        
        elif r==self.d-1 and c==0: # bottom left corner
            return [(r-1,c),(r,c+1)]
        
        elif r==self.d-1 and c==self.d-1: #bottom right corner
            return [(r,c-1),(r-1,c)]
        
        else:
            print("something went wrong!")
            return []
            
    def random_move(self):
        mat = self.board.copy()
        zero_pos = self.find_void()
        eval_moves = self.find_moves()
        next_pos = random.choice(eval_moves)
        self.board[zero_pos] = mat[next_pos] 
        self.board[next_pos] = 0
        return self.board
    
    def puzzle_shuffle(self,n):
        for _ in range(n):
            self.random_move()
        return self.board

In [13]:
'''
utility functions 
(some are duplicates of class' methods;
I know it is not optimal, but I do not have time 
to change the following complex functions 
to make them suit the class methods, 
that have been implemented later)
'''

def is_solved(mat,solution):
    if len(mat)>0:
        return (mat==solution).all()
    else:
        return False
    
def find_void(mat):
    pos = np.where(mat==0)
    return (pos[0].item(), pos[1].item())

def find_moves(mat):
    r,c = find_void(mat)
    dim = len(mat)-1
    
    if r in range(1,dim) and c in range(1,dim): # void cell not in contour
        return [(r,c-1),(r-1,c),(r+1,c),(r,c+1)]
    
    elif r in range(1,dim) and c==0: # first column, middle rows
        return [(r-1,c),(r+1,c),(r,c+1)]
    
    elif r in range(1,dim) and c==dim: # last column, middle rows
        return [(r,c-1),(r-1,c),(r+1,c)]
    
    elif r==0 and c in range(1,dim): # first row, middle columns
        return [(r,c-1),(r+1,c),(r,c+1)]
    
    elif r==dim and c in range(1,dim): # last row, middle columns
        return [(r,c-1),(r-1,c),(r,c+1)]
    
    elif r==0 and c==0: # upper left corner
        return [(r+1,c),(r,c+1)]
    
    elif r==0 and c==dim: # upper right corner
        return [(r+1,c),(r,c-1)]
    
    elif r==dim and c==0: # bottom left corner
        return [(r-1,c),(r,c+1)]
    
    elif r==dim and c==dim: #bottom right corner
        return [(r,c-1),(r-1,c)]
    
    else:
        print("something went wrong!")
        return []

# function used to verify is a puzzle configuration
# has already been visited   
def already_visited(mat,visited_nodes):
    if len(visited_nodes)==0:
        return False
    else:
        for state in visited_nodes:
            is_duplicate = (mat==state).all()
            if is_duplicate:
                break
        return is_duplicate
    
# function used to checkif a puzzle configuration
# is visited 2+ times in a path (like the solution)    
def any_duplicates(path):
    known_state = []
    doubles_count = 0
    for el in path:
        if not already_visited(el,known_state):
            if len(known_state)==0:
                known_state = el.copy()
            else:
                known_state = np.concatenate((el,known_state))
        else:
            doubles_count += 1
    return doubles_count, known_state

In [ ]:
# functions used to (try to) estimate the goodness of a move

# fitness 1: manhattan distance of first wrong element to its correct position
# (this fitness ended up being not good)
def puzzle_fitness(mat,solution):
    d1_board = mat.reshape(1,-1).squeeze()
    target=1 # we start looking for element 1
    
    for i in range(len(d1_board)):
        el = d1_board[i]
        if el == target:
            target += 1
        else:
            break
    
    if target == len(d1_board): #puzzle is solved => fitness = 0
        return 0
    else:
        a,b = np.where(mat==target) # position of the first wrong number
        c,d = np.where(solution==target) # right position of first wrong element
        fitness = abs(a-c) + abs(b-d)
        if fitness.size>0:
            return fitness.item()
        else:
            return 0

# fitness 2: manhattan distance of first wrong element to its correct position +
# + a term used to prevent not solving first current wrong element.
# (this fitness is also not good)
def firstWrong_fitness(mat,sol):
    d1_board = mat.reshape(1,-1).squeeze()
    d1_solution = sol.reshape(1,-1).squeeze()
    
    for i,j in zip(d1_board,d1_solution):
        if i != j:
            break
            
    if j:   # not solved
        target = j
        a,b = np.where(mat==target) # position of the first wrong number
        c,d = np.where(sol==target) # right position of first wrong element
        dist = abs(a-c) + abs(b-d)
        return len(d1_solution)*2*(len(mat)-1) + dist - j*2*(len(mat)-1)
    else:   # solved
        return 0

# sum of Manhattan Distances of ALL wrong pieces from their correct positions (zero included)
# (this is the function implemented in all solutions)
def manhattan_distance(mat,solution):
    dist = 0
    if not is_solved(mat,solution):
        d1_board = mat.reshape(1,-1).squeeze()
        d1_solution = solution.reshape(1,-1).squeeze()
        for i,j in zip(d1_board,d1_solution):
            if i != j:
                a,b = np.where(mat==j)  
                c,d = np.where(solution==j)
                man_dist = abs(a-c) + abs(b-d)
                dist += man_dist
        return dist.item()
    
    else:
        return dist
        
# this function outputs the distance from zero-cell to first wrong element.
# the main idea is that, when two moves share the same goodness, to tie-break
# we use this function, because, in order to move a wrong piece, we need to have
# zero-cell near it. (maybe this is useless, I don't know)
def zero_fitness(mat):
    d1_board = mat.reshape(1,-1).squeeze()
    target=1 # we satart looking for element 1
    
    for i in range(len(d1_board)):
        el = d1_board[i]
        if el == target:
            target += 1
        else:
            break
            
    a,b = np.where(mat==target) # position of the first wrong number  
    c,d = np.where(mat==0) # position of zero
    fitness = abs(a-c) + abs(b-d) 
    
    if fitness.size>0:
        return fitness.item()
    else:
        return 0

## Strategy 1: Bi-Directional Search

The idea of this method is to build two trees:
- One from the initial configuration (the shuffled puzzle)
- One from the solution

The trees are expanded computing, for each node, all possible moves,
unless they have already been visited once, in that tree.

When a common configuration is found in both tress, the solution path is computed 
concatenating the two trees on the common node (the one evolving from the solution is reversed, in order to have a proper path, from the initial configuration to the solution).

In [6]:
def find_path(mat, solution, MAX_STEPS=25):
    start = time.time()
    start_board = mat.copy()
    
    board_tree = [start_board]
    backward_tree = [solution]
    known_boards = []
    backward_boards = []

    if len(known_boards)==0:
        known_boards.append(start_board.tolist())
    if len(backward_boards)==0:
        backward_boards.append(solution.tolist())

    matched = False
    for i in tqdm(range(MAX_STEPS)):
        # expand forward tree, starting from initial configuration
        output = []
        for available_board in board_tree:
            if i == 0: # first iteration
                zero_pos = find_void(available_board)
                candidate_moves = find_moves(available_board)

                eval_boards = []
                for move in candidate_moves:
                    next_board = available_board.copy()
                    next_board[zero_pos] = available_board[move] 
                    next_board[move] = 0
                    if next_board.tolist() not in known_boards:
                        eval_boards.append(next_board)
                        known_boards.append(next_board.tolist())

                for board in eval_boards:
                    output.append(np.stack((available_board,board)))

            else: # we are on iteration 2+
                zero_pos = find_void(available_board[-1])
                candidate_moves = find_moves(available_board[-1])

                eval_boards = []
                for move in candidate_moves:
                    initial_board = available_board[-1].copy()
                    next_board = available_board[-1].copy()

                    next_board[zero_pos] = initial_board[move] 
                    next_board[move] = 0
                    if next_board.tolist() not in known_boards:
                        eval_boards.append(next_board)
                        known_boards.append(next_board.tolist())

                for board in eval_boards:
                    output.append(np.concatenate((available_board,board.reshape(1,start_board.shape[0],start_board.shape[1]))))

        if len(output)>0:
            board_tree = output.copy()

        # expand backward tree, starting from solution
        output = []
        for available_board in backward_tree:
            if i == 0:
                zero_pos = find_void(available_board)
                candidate_moves = find_moves(available_board)

                eval_boards = []
                for move in candidate_moves:
                    next_board = available_board.copy()

                    next_board[zero_pos] = available_board[move] 
                    next_board[move] = 0
                    if next_board.tolist() not in backward_boards:
                        eval_boards.append(next_board)
                        backward_boards.append(next_board.tolist())

                for board in eval_boards:
                    output.append(np.stack((available_board,board)))
                    
            else:
                zero_pos = find_void(available_board[-1])
                candidate_moves = find_moves(available_board[-1])

                eval_boards = []
                for move in candidate_moves:
                    initial_board = available_board[-1].copy()
                    next_board = available_board[-1].copy()

                    next_board[zero_pos] = initial_board[move] 
                    next_board[move] = 0
                    if next_board.tolist() not in backward_boards:
                        eval_boards.append(next_board)
                        backward_boards.append(next_board.tolist())

                for board in eval_boards:
                    output.append(np.concatenate((available_board,board.reshape(1,start_board.shape[0],start_board.shape[1]))))

        if len(output)>0:
            backward_tree = output.copy()

        # trees are expanded, now check for common node
        for f_path in board_tree: 
            for b_path in backward_tree:
                if f_path[-1].tolist() in b_path.tolist():
                    backward_tour = b_path
                    forward_tour = f_path
                    matched = True
                    print('Found a match between forward and backword trees!')
                if matched:
                    break # exiting from match-checking loops
        if matched:
            break # exiting from MAX_STEPS loop, since a solution path has been found
            
    end = time.time()
    if matched:
        # putting togheter forward and backward to create solution path        
        final_path = backward_tour.copy()
        complete = False

        while not complete:
            if final_path[-1].tolist() != forward_tour[-1].tolist():
                final_path = final_path[:-1]
            else:
                final_path = final_path[:-1]
                solution_path = np.concatenate((forward_tour, final_path[::-1]))
                complete = True
        print(f'Found a solution of {len(solution_path)} moves in {end-start:.3f} seconds')
        return solution_path
    
    # this is computed only when a match is not found in MAX_STEPS
    else:
        print('Solution not matched with initial board...')
        return start_board 

In [18]:
d = 3
puzzle = nPuzzle(d, n_shuffle=d**(2*(d-1)))
print(puzzle)

sol = find_path(puzzle.board,puzzle.solution)
#sol[-10:]

[[6 1 3]
 [0 5 2]
 [4 7 8]]


  0%|          | 0/25 [00:00<?, ?it/s]

Found a match between forward and backword trees!
Found a solution of 20 moves in 0.148 seconds


The algorithm can solve puzzle with d=3.
When d>=4, the trees' size explodes before finding a match.

## Strategy 2: Breadth first with limited depth and resets

Since the main issue with previous algorithm was the trees' size, we now aim to reduce the dimension of our problem.
- We build only one tree, starting from the suffled puzzle;
- We expand the tree using breadth-first strategy, but we limit the tree size, using MAX_DEPTH parameter.
- If the solution is not reached in MAX_DEPTH steps, we evaluate the best configuartion among the deepest node, using a specific function;
- We build a new tree, starting from the best configuration found;
- We repeat steps 3-4 until we find the solution (or until MAX_STEPS are reached, to stop the algorithm eventually...)
    - If , while expanding the tree, we cannot reach a new configuration (not already visited), we restart from step 1

In [19]:
# this function takes an initial board and expands it
# computing all possible steps, if they bring the configuration in a new state
# (a state not already visited). tree_depth is used to prevent growing too much the tree
# (this approach is compuationally very demanding for depth>10 and puzzle_dim>=4)

def make_tree_memory(mat, solution, tree_depth=7, prev_moves=[]):
    start_board = mat.copy()
    board_tree = [start_board]
    known_boards = prev_moves
    if len(known_boards)==0:
        known_boards.append(start_board.tolist())
    
    found_solution = is_solved(start_board,solution)
    if not found_solution:
        for i in range(tree_depth):
            output = []
            for available_board in board_tree: # evaluate all children nodes of a tree
                if i == 0: # first iteration: only one node (starting state)
                    zero_pos = find_void(available_board)
                    candidate_moves = find_moves(available_board)

                    eval_boards = []
                    for move in candidate_moves:
                        next_board = available_board.copy()
                        next_board[zero_pos] = available_board[move] 
                        next_board[move] = 0
                        
                        if next_board.tolist() not in known_boards:
                            eval_boards.append(next_board)
                            known_boards.append(next_board.tolist())

                    for board in eval_boards:
                        output.append(np.stack((available_board,board)))
                        if is_solved(board,solution):
                            break

                else: # we are on iteration 2+
                    zero_pos = find_void(available_board[-1])
                    candidate_moves = find_moves(available_board[-1])

                    eval_boards = []
                    for move in candidate_moves:
                        initial_board = available_board[-1].copy()
                        next_board = available_board[-1].copy()
                        next_board[zero_pos] = initial_board[move] 
                        next_board[move] = 0
                        
                        if next_board.tolist() not in known_boards:
                            eval_boards.append(next_board)
                            known_boards.append(next_board.tolist())

                    for board in eval_boards:
                        output.append(np.concatenate((available_board,board.reshape(1,start_board.shape[0],start_board.shape[1]))))
                        if is_solved(board,solution):
                            break

            board_tree = output # children nodes are all the ones generated in this step
            for path in board_tree:
                if is_solved(path[-1],solution): #checking if newly generated configuration solves puzzle
                    found_solution = True
                    break

            if found_solution:
                break
    return np.array(board_tree), known_boards


# this function is used to select the best branch of a tree.
# the criterion can be set by the user, when a tie is found
# a secondary function try to break it. 
# If tie not broken, choose randomly among bests.
def choose_best_tree(tree, solution, heuristic):
    if tree.shape == solution.shape:
        return tree
    
    else:
        fitness = float('inf')
        best_path = []
        
        for path in tree:           
            new_fit = heuristic(path[-1],solution) + len(path)
            if new_fit == fitness:
                best_path.append(path)

            elif new_fit < fitness:
                best_path = []
                best_path.append(path)
                fitness = new_fit

        if len(best_path) > 1: # if we have more than one possible path with lowest fitness
            list_optimal_path = []
            current_optimal = float('inf')
            for p in best_path:
                if zero_fitness(p[-1]) == current_optimal:
                    list_optimal_path.append(p)
                elif zero_fitness(p[-1]) < current_optimal:
                    current_optimal = zero_fitness(p[-1])
                    list_optimal_path = []
                    list_optimal_path.append(p)

            if len(list_optimal_path) == 1: # tie is broken
                return list_optimal_path[0]

            elif len(list_optimal_path) > 1:
                next_board = random.choice(list_optimal_path) # sorry, but at this point we choose randomly...
                return next_board

            else:
                print('List possible moves is empty... \nERROR!')
                return []

        elif len(best_path) == 1:
            return best_path[0]

        else:  # no path that leads to a better board configuration, according to heuristic
            return []

In [28]:
# solving puzzle

d=4
puzzle = nPuzzle(d, n_shuffle=d**(2*(d-1)))
solution = puzzle.solution
board = puzzle.board.copy()
print(f'Initial Board:\n {board}')

MAX_STEPS = 5000
TREE_DEPTH = 7
known_moves = []
total_path = []
n_resets= 0

start = time.time()
current_board = board.copy()
for _ in tqdm(range(MAX_STEPS), desc='Solving'):
        
    if not is_solved(current_board,solution):

        # expand tree
        step_trees, step_moves = make_tree_memory(mat=current_board, solution=solution, prev_moves=known_moves, tree_depth=TREE_DEPTH)
        if len(step_moves)>0:
            known_moves.append(step_moves)

        # evaluate best tree, and build solution path
        current_tree = choose_best_tree(tree=step_trees, solution=solution, heuristic=manhattan_distance)
        if len(current_tree) > 0:
            if len(total_path)==0:
                total_path = current_tree.copy()
            else:
                total_path = np.concatenate((total_path,current_tree))
            current_board = total_path[-1]  
        # if no new nodes are found -> restart from initial configuration    
        else:
            n_resets += 1
            current_board = board.copy()
            known_moves = []
            
    else:
        end = time.time()
        print(f'Found a solution of {len(total_path)} moves in {end-start:.3f} seconds')
        break
        
if not is_solved(total_path[-1],solution):
    print('Puzzle not solved')

Initial Board:
 [[ 9  8 12 15]
 [11  1 14  5]
 [ 3  7  6 13]
 [10  0  4  2]]


Solving:   0%|          | 0/5000 [00:00<?, ?it/s]

Found a solution of 4595 moves in 80.497 seconds


In [ ]:
# inspect solution path
total_path[-25:]

array([[[ 1,  2,  4,  8],
        [ 5,  6,  0,  3],
        [14,  9,  7, 11],
        [13, 10, 15, 12]],

       [[ 1,  2,  4,  8],
        [ 5,  6,  3,  0],
        [14,  9,  7, 11],
        [13, 10, 15, 12]],

       [[ 1,  2,  4,  0],
        [ 5,  6,  3,  8],
        [14,  9,  7, 11],
        [13, 10, 15, 12]],

       [[ 1,  2,  0,  4],
        [ 5,  6,  3,  8],
        [14,  9,  7, 11],
        [13, 10, 15, 12]],

       [[ 1,  2,  3,  4],
        [ 5,  6,  0,  8],
        [14,  9,  7, 11],
        [13, 10, 15, 12]],

       [[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [14,  9,  0, 11],
        [13, 10, 15, 12]],

       [[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [14,  9,  0, 11],
        [13, 10, 15, 12]],

       [[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [14,  0,  9, 11],
        [13, 10, 15, 12]],

       [[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 0, 14,  9, 11],
        [13, 10, 15, 12]],

       [[ 1,  2,  3,  4],
        [ 5,  6,  7,

'''
The algorithm is able to solve Puzzle with d<=4, in a reasonable amount of time.
When d=4, it is slow (1-5 minutes) but it will eventually find the solution.
'''